In [4]:
import os
os.chdir('../')

In [5]:
%pwd

'c:\\Ml_Ops_Pro'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass 
class TrainingConfig:
    root_dir: Path
    trained_model: Path
    update_base_model_path: Path
    training_data: Path
    prams_epochs: int
    params_batch_size: int
    params_agumentation: int
    params_img_size: list



In [7]:
from src.CnnClasifier.constants import *
from src.CnnClasifier.utils.common import read_yaml, create_directories

class ConfigManeger:
    def __init__(self,
                 config_filepath = path_of_config,
                 params_filepat = path_of_params):
        self.config =  read_yaml(config_filepath)
        self.params = read_yaml(params_filepat)

        create_directories([self.config.artifacts_root])


    def get_tarinig_config(self) ->TrainingConfig:
        training = self.config.model_training
        prepare_base_model = self.config.prepare_base_model

        params = self.params

        training_data = os.path.join(self.config.data_ingestion.unzip_dir, 'kidney-ct-scan-image')

        create_directories([Path(training.root_dir)])

        tarining_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model= Path(training.trained_model_path),
            update_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data =  Path(training_data),
            prams_epochs= params.EPOCHS,
            params_batch_size= params.BATCH_SIZE,
            params_agumentation= params.AUGMENTATION,
            params_img_size=params.IMG_SIZE

        )
        return tarining_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from PIL import Image

In [14]:
from PIL import Image
from tensorflow.keras.optimizers import Adam

class Training:
    def __init__(self, config: TrainingConfig):
         self.config = config

    def get_base_model(self):
         self.model = tf.keras.models.load_model(
              self.config.update_base_model_path
         )
    def train_valid_generator(self):
         datagen_kward = dict(
               rescale = 1./255,
            validation_split=0.20
         )

         dataflow_kwards = dict(
              
          #     print(self.config.params_img_size[:, -1]),

              
              target_size = self.config.params_img_size[:-1],
              batch_size = self.config.params_batch_size,
              interpolation = 'bilinear'
         )

         valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
              **datagen_kward
         )

         self.valid_generator = valid_datagen.flow_from_directory(
              directory = self.config.training_data,
              subset = 'validation',
              shuffle = False,
              **dataflow_kwards
         )

         if self.config.params_agumentation:
              train_datagenearator = tf.keras.preprocessing.image.ImageDataGenerator(
                   rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagen_kward
              )
         else:
              train_datagenearator = valid_datagen


         self.train_generator = train_datagenearator.flow_from_directory(
              directory=self.config.training_data,
                          subset="training",
                          shuffle=True,
                          **dataflow_kwards
         )
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
         model.save(path)
        

    def train(self):
     #    self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
     #    self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        optimizer = Adam(learning_rate=0.001)


        self.model.compile(optimizer=optimizer)

        self.model.fit(
             self.train_generator,
             epochs = self.config.prams_epochs,
          #    steps_per_epoch=self.steps_per_epoch
          #    validation_steps=self.validation_steps,
            validation_data = self.valid_generator,
        )

        self.save_model(
             path= self.config.trained_model,
             model= self.model
        )

In [15]:
try:
    conf = ConfigManeger()
    training_confi = conf.get_tarinig_config()
    training = Training(config=training_confi)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
     raise e

[2024-11-01 15:21:44,928: INFO: common: yaml file config\config.yaml is loaded succesfully]
[2024-11-01 15:21:44,932: INFO: common: yaml file params.yaml is loaded succesfully]
[2024-11-01 15:21:44,934: INFO: common: created directory at: artifacts]
[2024-11-01 15:21:44,934: INFO: common: created directory at: artifacts\training]
[2024-11-01 15:21:45,304: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.


c:\Ml_Ops_Pro\myenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
 5/24 ━━━━━━━━━━━━━━━━━━━━ 1:00 3s/step - accuracy: 0.5564 - loss: 0.7665

KeyboardInterrupt: 